# Streamlined NTRU Prime

O desenvolvimento do Streamlined NTRU Prime está dividido nos seguintes passos:

1. Geração dos parâmetros e anéis de polinómios
2. Declaração das funções auxiliares necessárias
3. Especificação da classe SNTRUP
4. Exemplo de utilização

## Parâmetros e geração dos anéis de polinómios

Analisando as várias possibilidades para inicialização dos parâmetros, disponibilizadas no documento da candidatura NTRU Prime, optou-se pela especificação `kem/sntrup4591761`.

In [1]:
p = 761
q = 4591
w = 286
 
Zx.<x>  = ZZ[]
Z3.<y>  = PolynomialRing(GF(3))
Gq.<z>  = GF(q)[]
 
R.<x> = Zx.quotient(x^p-x-1)  # R
R3.<y> = Z3.quotient(y^p-y-1) # R/3
Rq.<z> = Gq.quotient(z^p-z-1) # R/q

###  Funções auxiliares

In [2]:
from random import choice, randint

def _small_poly(p,t=None):
    if not t:
        return Zx([choice([-1,0,1]) for k in range(p)])
    l = [choice([-1,1]) for k in range(p)]
    u = p-2*t
    count = 0
    while count < u:
        k = randint(0,p-1)
        while l[k] == 0:
            k = randint(0, p-1)
        l[k] = 0
        count += 1
    return Zx(l)


def _center_lift(x):
    return lift(x + q//2) - q//2

def _round(w,n=q):          
    """
         input:  polinómio em Rq ou R3
         output: transpõe os coeficientes para o intervalo -n//2..+n//2
    """
    return Zx(map(lambda x: _center_lift(x), w.list()))
    
def _round_3(w):        
    """ 
         transpõe os coeficientes de "w" para o intervalo -q//2..+q//2 
         e arredonda-os ao múltiplo de 3 mais próximo
    """
    def _f(x):
        return 3*round(x/3)
    return  Zx(map(lambda x: _f(_center_lift(x)) , w.list()))


def _round_3_to_small(w):
    def _f(x):
        return x - 3*round(x/3)
    return  Zx(map(lambda x: _f(lift(x)) , w.list()))
    


## Função de Hash

Tendo-se escolhido a especificação `kem/sntrup4591761`, então a função de hash a utilizar tem de ser SHA-512.

In [3]:
import hashlib
def hash(s):h = hashlib.sha512(); h.update(s); return h.digest()

**Teste e verificação**

In [4]:
import itertools

def concat(lists): return list(itertools.chain.from_iterable(lists))

def int2str(u,bytes):
    return ''.join([chr((u//256^i)%256) for i in range(bytes)])

def str2int(s):
    return sum([ord(s[i])*256^i for i in range(len(s))])

def seq2str(u,radix,batch,bytes):
    return ''.join(int2str(sum(u[i+t]*radix^t for t in range(batch)),bytes)
                 for i in range(0,len(u),batch))

def str2seq(s,radix,batch,bytes):
    u = [str2int(s[i:i+bytes]) for i in range(0,len(s),bytes)]
    return concat([(u[i]//radix^j)%radix for j in range(batch)]\
                  for i in range(len(u)))

def encodeZx(m):
    m = [m[i]+1 for i in range(p)] + [0]*3
    return seq2str(m,4,4,1)

def decodeZx(mstr):
    m = str2seq(mstr,4,4,1)
    return Zx([m[i]-1 for i in range(p)])

def encodeRq(h):
    h = [q//2 + _center_lift(h[i]) for i in range(p)] + [0]*4
    return seq2str(h,6144,5,8)[:1218]

def decodeRq(hstr):
    h = str2seq(hstr,6144,5,8)
    if max(h) >= q: raise Exception("pk out of range")
    return Rq([h[i]-q//2 for i in range(p)])

def encoderoundedRq(c):
    # 2295 / 3 = 765
    c = [765 + _center_lift(c[i]/3) for i in range(p)] + [0]
    return seq2str(c,1536,3,4)[:1015]

def decoderoundedRq(cstr):
    c = str2seq(cstr,1536,3,4)
    if max(c) > 765*2: raise Exception("c out of range")
    return 3*Rq([c[i]-765 for i in range(p)])

## Classe SNTRUP

A classe SNTRUP implementa a versão `kem/sntrup4591761` e sendo um KEM está dividida entre setup (`__init__`), `encapsulate` e `decapsulate`.

### Setup

A função `__init__` está responsável por realizar a geração das chaves utilizadas no NTRU. Assim sendo, o algoritmo segue os seguintes passos:

1. **Geração uniforme aleatória de um elemento pequeno $g \in R$, até que este seja invertível em $R/3$**.
    - Ao correr a função `small_poly` o código que está a ser executado é o seguinte: `Zx([choice([-1,0,1]) for k in range(p)])`. Na prática está a ser gerado um vetor onde se insere o elemento `-1`, `0` ou `1` aleatoriamente. Estes por sua vez são os únicos coeficientes permitidos para elementos pequenos. Antes de testar se o elemento é invertível através da função `is_unit` é necessário convertê-lo para $R/3$ através do `R3`.
    
    
2. **Geração uniforme aleatória de um elemento pequeno $f \in R$, com peso `w`.**
    - Este passo utiliza na mesma a função `small_poly` mas desta vez é necessário gerar um elemento com peso `w`, ou seja, `w` coeficientes devem ser diferentes de 0. Para isso começa-se por preencher todos os coeficientes a 0 e depois vai-se preenchendo aleatoriamente `w` posições com `-1` ou `1`.
    
    
3. **Calcular $h = g/(3f)$ em $R/q$.**
    - Dado que o cálculo precisava de ser realizado em $R/q$, então recorreu-se ao `Rq` para a conversão.
    
    
4. **Codificação da chave pública `h` numa string.**
    - Na prática significa fazer o `lift` de cada elemento para que fique dentro do intervalo {-q//2, ..., q//2} e, posteriormente, somar q//2 o que faz mover o intervalo para {0, ..., q-1}.
    
    `[q//2 + _center_lift(h[i]) for i in range(p)]`
    
    - É também dito que os últimos 4 elementos são sempre zero.
    
    `h = [q//2 + _center_lift(h[i]) for i in range(p)] + [0]*4`
                          
    - De seguida, chama-se a função `seq2str` que percorre os coeficientes em grupos de 5 e produz um inteiro com a seguinte estrutura: $$c_0 + 6144c_1 + 6144^2c_2 + 6144^3c_3 + 6144^4c_4$$ em código: `sum(u[i+t]*6144^t for t in range(5)`
    
    - Por fim a função `int2str` converte o inteiro para string de 8 bytes na forma de *litte-endian*.
    
    
5. **Guarda-se a chave privada: $f$ em $R$ e $1/g$ em $R/3$.**
    - Estando o `f` já calculado, basta calcular o $1/g$ da seguinte forma: `R3(g)^(-1)`.
    

### Encapsulate

A função `encapsulate` é responsável por emular a geração de um criptograma por parte de um emissor. Este processo é descrito pelos seguintes passos:

1. **Descoficiar a string da chave pública de forma a obter o $h \in R/q$.**
    - Para isso utiliza-se a função `decodeRq` que implementa precisamente o processo inverso à função `encodeRq`.
    
2. **Geração uniforme aleatória de um elemento pequeno $f \in R$, com peso `w`.**
    - Processo identico á geração da chave privada na função `__init__`.
    
3. **Cálculo do $hr \in R/q$.**
    - O código é direto: `hr = h * Rq(r)`
    
4. **Arredondar cada coeficiente do `hr` para um inteiro entre -q//2 e q//2 e arredonda-o ao múltiplo de 3 mais próximo.**
    - Este processo é conseguido através da função `_round_3`. Esta função começa por fazer um `lift` centrado de forma a que os valores estejam contidos no intervalo estipulado: `Zx(map(lambda x: _center_lift(x), w.list()))`. De seguida é necessário arredondar os valores para o múltiplo de 3 mais próximo: `3*round(x/3)`. 
    
    - No fim tem-se: `Zx(map(lambda x: 3*round(_center_lift(x)/3), w.list()))`
    
    
5. **Codificação dp criptograma `c` numa string.**
    - Na prática significa fazer o `lift` de cada elemento para que fique dentro do intervalo {-q//2, ..., q//2} e, posteriormente, somar (q//2)/3 (=765) o que faz mover o intervalo para {0, ..., (q//2)-(q//2)/3}.
    
    `[765 + _center_lift(c[i]/3) for i in range(p)]`
    
    - É também dito que o último elemento é sempre zero.
    
    `c = [765 + _center_lift(c[i]/3) for i in range(p)] + [0]`
                          
    - De seguida, chama-se a função `seq2str` que percorre os coeficientes em grupos de 3 e produz um inteiro com a seguinte estrutura: $$c_0 + 1536c_1 + 1536^2c_2$$ em código: `sum(u[i+t]*1536^t for t in range(3)`
    
    - Por fim a função `int2str` converte o inteiro para string de 4 bytes na forma de *litte-endian*.
    
  
6. **Cálculo do valor de hash do `r`.**
    - A documentação diz que para calcular usar a função de hash é necessário que cada coeficiente do polinómio seja incrementado numa unidade: `[m[i]+1 for i in range(p)]`. 
    
    - Além disso os últimos 3 elementos são sempre 0: 
    
    `m = [m[i]+1 for i in range(p)] + [0]*3`
    
    - De seguida, chama-se a função `seq2str` que percorre os coeficientes em grupos de 4 e produz um inteiro com a seguinte estrutura: $$c_0 + 4c_1 + 4^2c_2 + 4^3c_3$$ em código: `sum(u[i+t]*4^t for t in range(4)`.
    
    - Por fim a função `int2str` converte o inteiro para string de 1 byte. Os primeiros 32 bytes são o valor para confirmação, enquanto que os segundos são a chave de sessão.
    

### Decapsulate

1. **Descoficiar a string do criptograma de forma a obter o $c \in R$.**
    - Para isso utiliza-se a função `decoderoundedRq` que implementa precisamente o processo inverso à função `encoderoundedRq`.


2. **Multiplicar o $c$ por $3f$ em $R/q$.**
    - O código é direto: `f3mgr = Rq(3*f) * c`
    
    
3. **Ver cada coeficiente de $3fc$ em $R/q$ num inteiro entre -q//2 e q//2, e reduzir módulo 3 para obter um polinómio em $R/3$.**
    - A primeira tarefa é cumprida através da chamda à função `_round`: `f3mgr = _round(f3mgr)`.
    - Para a segunda parte basta: `R3(f3mgr)`.
    
  
4. **Multiplicar por $1/g$ em $R/3$.**
    - O código é direto: `r = R3(s) * R3(f3mgr)`.
    

5. **Levantar a multiplicação anterior em $R/3$ para um polinómio reduzido $r' \in R$.**
    - Primeiramente é realizado o lift a cada um dos elementos do `r`: `Zx(map(lambda x: lift(x), w.list()))`. Neste momento passou-se de $R/3$ para $R$ no entanto os valores do coeficientes encontram-se compreendidos no intervalo {-3, ..., 3}. Para ser um polinómio reduzido é necessário que os coeficientes sejam um dos seguintes valores {-1, 0, 1}. Para tal é necessário realizar uma operação adicional que vai originar o código final: `Zx(map(lambda x: lift(x) - 3*round(lift(x)/3) , w.list()))`
    
    
6. **Cálculo do $c'$, $C'$, $K'$ a partir de $r'$ como na função de `encapsulate`.**
    - No fim se o r for reduzido, tiver um peso w, o `new_c = c` e o `confirm = new_k[:32]` então deve ser retornada a K', ou seja, a `new_k[32:]`. Caso contrário é returnado `False`.

In [5]:
class SNTRUP(object):
    def __init__(self):
        # 1
        g = _small_poly(p)
        while not R3(g).is_unit():
            g = _small_poly(p)
        # 2
        f = _small_poly(p,w)
        # 3
        h = Rq(g)/Rq(3*f)
        # 4
        self.pk = encodeRq(h)           # chave pública
        # 5
        self.secret = (f , R3(g)^(-1))  # chave privada
        
        
    def encapsulate(self):
        # 1
        h = decodeRq(self.pk)
        # 2
        r = _small_poly(p,w)
        # 3
        hr = h * Rq(r)
        # 4
        c = Rq(_round_3(hr))
        # 6
        key = hash(encodeZx(r))
        # 5 - encode c
        return (key[:32] + encoderoundedRq(c), key[32:])
 
    def decapsulate(self, Cc):
        (f , s) = self.secret
        h = decodeRq(self.pk)
        # 1 - decode c
        confirm, c = Cc[:32], decoderoundedRq(Cc[32:])
        # 2
        f3mgr = Rq(3*f) * c
        # 3
        f3mgr = _round(f3mgr)
        # 4
        r = R3(s) * R3(f3mgr)
        # 5
        r = _round_3_to_small(r)
        # 6 - compute c'
        hr = h * Rq(r)
        new_c = Rq(_round_3(hr))
        # 6 - compute C' and K'
        new_k = hash(encodeZx(r))
        if sum(r[i]==0 for i in range(p)) != p-2*w: return False
        if new_c != c: return False
        if new_k[:32] != confirm: return False
        return new_k[32:]

## Teste e Verificação

Por fim são realizados diversos testes que verificam se a implementação do SNTRUP corresponde ao esperado. Se tudo correr bem deve ser imprimido no ecrã a palavra 'DONE'.

In [6]:
for keys in range(5):
    K = SNTRUP()
    for ciphertexts in range(5):
        (Cc,k) = K.encapsulate() 
        assert K.decapsulate(Cc) == k
print 'DONE'

DONE
